In [1]:
# install pymongo if not installed

#! pip install pymongo==3.11.3

In [83]:
from pymongo import MongoClient
import pandas as pd

# connect to local mongodb and database test
 
MONGO_HOST= 'mongodb://localhost:27017/test'

client = MongoClient(MONGO_HOST)

# select database
db=client['test']

#select the collection within the database
collection = db.healthImpact

In [84]:
# disply first document in the collection
result=collection.find()
result[0]

{'_id': ObjectId('6217cd460af6ac5787de0048'),
 'created_at': 'Thu Feb 24 18:24:01 +0000 2022',
 'id': 1496913822854140000,
 'id_str': '1496913822854139908',
 'text': 'RT @DrBrendaMTR: Love this! Sleep is vital for optimal for human health … human health and environmental health are interconnected, sleep m…',
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 115105845,
  'id_str': '115105845',
  'name': 'Liz Borkowski',
  'screen_name': 'LizBorkowski',
  'location': 'DC',
  'url': 'http://www.thepumphandle.org/',
  'description': "Public health researcher at George Washington University, managing editor of Women's Health Issues. She/her, views are my own.",
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_

In [85]:
# create a pipeline to extract certain fields

pipeline =[
   {"$project": {
   "tweet_id": "$id_str",
    "created_at":1,
    "user_name":"$user.name",
    "user_screenName": "$user.screen_name",
    "text":1,
    "source":1,
    "place":1,
     "lang":1,
     "_id":0
  }
  }
]

result=collection.aggregate(pipeline)

result_list=[]  # create an array to store the result

for res in result:
    #print(res)
    result_list.append(res)
    
result_list

# convert result list into pandas data frame

result_pd = pd.DataFrame(list(result_list))

result_pd.head()

,created_at,text,source,lang,tweet_id,user_name,user_screenName,place
0,Thu Feb 24 18:24:01 +0000 2022,"RT @DrBrendaMTR: Love this! Sleep is vital for optimal for human health … human health and environmental health are interconnected, sleep m…","<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",en,1496913822854139908,Liz Borkowski,LizBorkowski,NaN
1,Mon Feb 28 10:03:27 +0000 2022,"RT @J_YouthAdvocate: Plastics threatens ocean health, the health of marine species, food safety and quality, human health, coastal tourism,…","<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",en,1498237401554108416,Joseph Macharia Kimemia,J_YouthAdvocate,NaN
2,Mon Feb 28 10:01:17 +0000 2022,"RT @J_YouthAdvocate: Plastics threatens ocean health, the health of marine species, food safety and quality, human health, coastal tourism,…","<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",en,1498236858060398592,Faith Nabwire,FaithNabwire12,NaN
3,Sun Feb 27 13:58:58 +0000 2022,RT @thewyliekate1: Climate change is a health issue. Health professionals need to advocate for climate action to protect human health. By j…,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",en,1497934285071732737,Sue Cooke,SMC54,NaN
4,Sun Feb 27 09:15:19 +0000 2022,RT @thewyliekate1: Climate change is a health issue. Health professionals need to advocate for climate action to protect human health. By j…,"<a href=""https://Eco1stArt.com"" rel=""nofollow"">Eco Promotions</a>",en,1497862903860539394,Andrea,Eco1stArt,NaN


## Sentiment Analysis

In this next block of code we will cycle through all the tweets and score the tweet text using the VADER Sentiment library. See: https://pypi.org/project/vaderSentiment/

In [ ]:
#! pip install vaderSentiment==3.3.2

In [86]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sentiment = []
analyzer = SentimentIntensityAnalyzer()

for index, row in result_pd.iterrows():
    tweet = str(row['text'])
    vs = analyzer.polarity_scores(tweet)
    sentiment.append(vs)
    
result_pd['VaderSentiment'] = sentiment

In [87]:
result_pd.head()

,created_at,text,source,lang,tweet_id,user_name,user_screenName,place,VaderSentiment
0,Thu Feb 24 18:24:01 +0000 2022,"RT @DrBrendaMTR: Love this! Sleep is vital for optimal for human health … human health and environmental health are interconnected, sleep m…","<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",en,1496913822854139908,Liz Borkowski,LizBorkowski,NaN,"{'neg': 0.0, 'neu': 0.674, 'pos': 0.326, 'compound': 0.8478}"
1,Mon Feb 28 10:03:27 +0000 2022,"RT @J_YouthAdvocate: Plastics threatens ocean health, the health of marine species, food safety and quality, human health, coastal tourism,…","<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",en,1498237401554108416,Joseph Macharia Kimemia,J_YouthAdvocate,NaN,"{'neg': 0.116, 'neu': 0.759, 'pos': 0.125, 'compound': 0.0516}"
2,Mon Feb 28 10:01:17 +0000 2022,"RT @J_YouthAdvocate: Plastics threatens ocean health, the health of marine species, food safety and quality, human health, coastal tourism,…","<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",en,1498236858060398592,Faith Nabwire,FaithNabwire12,NaN,"{'neg': 0.116, 'neu': 0.759, 'pos': 0.125, 'compound': 0.0516}"
3,Sun Feb 27 13:58:58 +0000 2022,RT @thewyliekate1: Climate change is a health issue. Health professionals need to advocate for climate action to protect human health. By j…,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",en,1497934285071732737,Sue Cooke,SMC54,NaN,"{'neg': 0.0, 'neu': 0.89, 'pos': 0.11, 'compound': 0.3818}"
4,Sun Feb 27 09:15:19 +0000 2022,RT @thewyliekate1: Climate change is a health issue. Health professionals need to advocate for climate action to protect human health. By j…,"<a href=""https://Eco1stArt.com"" rel=""nofollow"">Eco Promotions</a>",en,1497862903860539394,Andrea,Eco1stArt,NaN,"{'neg': 0.0, 'neu': 0.89, 'pos': 0.11, 'compound': 0.3818}"


In [88]:
# Now let's change the setting so a print will show all of the columns, change the column max width so we can see the whole
# tweet, and show up to 100 rows
pd.set_option('display.max_columns', None) # This will make it so all columns appear when printing
pd.options.display.max_colwidth = 300  # This makes it so we can see the entire tweet text.
pd.set_option('display.max_rows', 10)

In [17]:
result_pd.head()

,created_at,text,source,lang,tweet_id,user_name,user_screenName,place,VaderSentiment
0,Thu Feb 24 18:24:01 +0000 2022,"RT @DrBrendaMTR: Love this! Sleep is vital for optimal for human health … human health and environmental health are interconnected, sleep m…","<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",en,1496913822854139908,Liz Borkowski,LizBorkowski,NaN,"{'neg': 0.0, 'neu': 0.674, 'pos': 0.326, 'compound': 0.8478}"
1,Mon Feb 28 10:03:27 +0000 2022,"RT @J_YouthAdvocate: Plastics threatens ocean health, the health of marine species, food safety and quality, human health, coastal tourism,…","<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",en,1498237401554108416,Joseph Macharia Kimemia,J_YouthAdvocate,NaN,"{'neg': 0.116, 'neu': 0.759, 'pos': 0.125, 'compound': 0.0516}"
2,Mon Feb 28 10:01:17 +0000 2022,"RT @J_YouthAdvocate: Plastics threatens ocean health, the health of marine species, food safety and quality, human health, coastal tourism,…","<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",en,1498236858060398592,Faith Nabwire,FaithNabwire12,NaN,"{'neg': 0.116, 'neu': 0.759, 'pos': 0.125, 'compound': 0.0516}"
3,Sun Feb 27 13:58:58 +0000 2022,RT @thewyliekate1: Climate change is a health issue. Health professionals need to advocate for climate action to protect human health. By j…,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",en,1497934285071732737,Sue Cooke,SMC54,NaN,"{'neg': 0.0, 'neu': 0.89, 'pos': 0.11, 'compound': 0.3818}"
4,Sun Feb 27 09:15:19 +0000 2022,RT @thewyliekate1: Climate change is a health issue. Health professionals need to advocate for climate action to protect human health. By j…,"<a href=""https://Eco1stArt.com"" rel=""nofollow"">Eco Promotions</a>",en,1497862903860539394,Andrea,Eco1stArt,NaN,"{'neg': 0.0, 'neu': 0.89, 'pos': 0.11, 'compound': 0.3818}"


In [1]:
# extract compound value for the VaderSentiment



In [3]:
# Classify compund score into positive, neutral and negative

In [2]:
# visualize the result


# Gender Guesser

In this section we will guess a persons gender based off of their first name. We split the name using the first space found into a first and last name.

In [58]:
# You only need to run this once then comment out the code.
!pip install gender_guesser

In [4]:
# This splits the name field into two different fields and saves it in a new dataframe


In [97]:
# This creates a new field in the original dataframe using the first column in the names dataframe



In [98]:
import gender_guesser.detector as gender

d = gender.Detector()

In [6]:
# DONT RUN THIS WITH YOUR FULL DATA!!! THIS IS FOR DEMO PURPOSES ONLY AND DOES NOT HAVE TO BE RUN
# Here we are looping through all the records and displaying the first name along with the "guessed" gender

In [5]:
# Let's rerun it and save the guessed gender back to the dataframe


In [7]:
# let create a boxplot to see whether sentiment differ by gender


In [103]:
# save the result in a csv file at current directoy


In [104]:
# read the save 

df=pd.read_csv("healthImpact.csv", sep="|")

In [82]:
df.head()

,created_at,text,source,lang,tweet_id,user_name,user_screenName,place,VaderSentiment,compound,sentiment,First,Gender,gender
0,Thu Feb 24 18:24:01 +0000 2022,"RT @DrBrendaMTR: Love this! Sleep is vital for optimal for human health … human health and environmental health are interconnected, sleep m…","<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",en,1496913822854139908,Liz Borkowski,LizBorkowski,NaN,"{'neg': 0.0, 'neu': 0.674, 'pos': 0.326, 'compound': 0.8478}",0.8478,Positive,Liz,female,female
1,Mon Feb 28 10:03:27 +0000 2022,"RT @J_YouthAdvocate: Plastics threatens ocean health, the health of marine species, food safety and quality, human health, coastal tourism,…","<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",en,1498237401554108416,Joseph Macharia Kimemia,J_YouthAdvocate,NaN,"{'neg': 0.116, 'neu': 0.759, 'pos': 0.125, 'compound': 0.0516}",0.0516,Positive,Joseph,male,male
2,Mon Feb 28 10:01:17 +0000 2022,"RT @J_YouthAdvocate: Plastics threatens ocean health, the health of marine species, food safety and quality, human health, coastal tourism,…","<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",en,1498236858060398592,Faith Nabwire,FaithNabwire12,NaN,"{'neg': 0.116, 'neu': 0.759, 'pos': 0.125, 'compound': 0.0516}",0.0516,Positive,Faith,female,female
3,Sun Feb 27 13:58:58 +0000 2022,RT @thewyliekate1: Climate change is a health issue. Health professionals need to advocate for climate action to protect human health. By j…,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",en,1497934285071732737,Sue Cooke,SMC54,NaN,"{'neg': 0.0, 'neu': 0.89, 'pos': 0.11, 'compound': 0.3818}",0.3818,Positive,Sue,female,female
4,Sun Feb 27 09:15:19 +0000 2022,RT @thewyliekate1: Climate change is a health issue. Health professionals need to advocate for climate action to protect human health. By j…,"<a href=""https://Eco1stArt.com"" rel=""nofollow"">Eco Promotions</a>",en,1497862903860539394,Andrea,Eco1stArt,NaN,"{'neg': 0.0, 'neu': 0.89, 'pos': 0.11, 'compound': 0.3818}",0.3818,Positive,Andrea,female,female
